## 1. Word2Vec

**Word2Vec**은 단어를 고정된 차원의 밀집 벡터(dense vector)로 표현하는 분산 표현(distributed representation) 방법 중 하나입니다. Word2Vec은 단어의 의미를 벡터 공간에 투영하여, 의미가 비슷한 단어들은 벡터 공간에서 가깝게 위치하도록 학습합니다.

**Word2Vec의 핵심 아이디어:**

*   **분포 가설 (Distributional Hypothesis):** "비슷한 문맥에서 나타나는 단어는 비슷한 의미를 가진다."
*   Word2Vec은 이 가설을 바탕으로, 대량의 텍스트 데이터(코퍼스)를 학습하여 단어 간의 관계를 파악하고, 각 단어를 벡터로 표현합니다.

**Word2Vec의 장점:**

*   **단어 간의 의미 관계 파악:** 벡터 연산을 통해 단어 간의 유사도, 관계 등을 파악할 수 있습니다. (예: "왕" - "남자" + "여자" ≈ "여왕")
*   **차원 축소:** 고차원의 희소 벡터(예: one-hot encoding)를 저차원의 밀집 벡터로 변환하여 계산 효율성을 높입니다.
*   **전이 학습 (Transfer Learning):** Word2Vec으로 학습된 임베딩(embedding)을 다른 자연어 처리 작업(예: 텍스트 분류, 감성 분석)의 초기값으로 사용하여 성능을 향상시킬 수 있습니다.

**Word2Vec의 종류:** CBOW, Skip-gram (아래에서 설명)

## 2. CBOW와 Skip-gram

Word2Vec은 크게 두 가지 모델로 나뉩니다.

**CBOW (Continuous Bag-of-Words):**

*   **주변 단어 -> 중심 단어 예측:** 주변 단어(context words)들을 입력으로 받아 중심 단어(target word)를 예측하는 방식으로 학습합니다.
*   **구조:**
    *   입력층(Input Layer): 주변 단어들의 one-hot encoding 벡터
    *   투영층(Projection Layer): 입력 벡터들을 평균내어 은닉층(hidden layer)에 투영 (가중치 행렬 곱)
    *   출력층(Output Layer): 중심 단어를 예측하는 확률 분포 (softmax)
*   **특징:**
    *   학습 속도가 빠릅니다.
    *   상대적으로 빈번하게 등장하는 단어에 대해 더 잘 학습합니다.

**Skip-gram:**

*   **중심 단어 -> 주변 단어 예측:** 중심 단어를 입력으로 받아 주변 단어들을 예측하는 방식으로 학습합니다.
*   **구조:**
    *   입력층: 중심 단어의 one-hot encoding 벡터
    *   투영층: 입력 벡터를 은닉층에 투영 (가중치 행렬 곱)
    *   출력층: 주변 단어들을 예측하는 확률 분포 (softmax)
*   **특징:**
    *   CBOW보다 학습 속도는 느리지만, 드물게 등장하는 단어나 특정 패턴을 가진 단어에 대해 더 잘 학습하는 경향이 있습니다.
    *   일반적으로 CBOW보다 성능이 좋은 경우가 많습니다.

**CBOW vs. Skip-gram 요약:**

| 특징         | CBOW                                                               | Skip-gram                                                     |
| :----------- | :----------------------------------------------------------------- | :------------------------------------------------------------ |
| **예측 방향** | 주변 단어 -> 중심 단어                                               | 중심 단어 -> 주변 단어                                           |
| **학습 속도** | 빠름                                                              | 느림                                                           |
| **성능**      | 빈번한 단어에 강함                                                | 드문 단어, 특정 패턴에 강함, 일반적으로 CBOW보다 성능이 좋은 경우가 많음 |

## 3. 영화 시놉시스 데이터와 Word2Vec을 이용한 영화별 토큰 평균 벡터 계산

```python
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 이전 코드에서 preprocess_synopsis 함수와 영화 데이터(movies)를 가져와서 사용
# 영화 데이터 (시놉시스, 장르, 배우)
movies = [
    {
        "synopsis": """
        "나한테 별로 고마워하지 않아도 돼요" 까칠한 어른 윤서
        "한 번 쯤은 자기를 믿어주는 사람이 있으면 좋잖아요" 꿈 없는 청년 수찬

        시청 정기간행물의 인터뷰어 '윤서'에게 사람의 온기는 한여름의 습하고 불쾌한 더위 같은 것.
        그러던 어느 날, 청년 배달원 '수찬'과 실랑이를 벌이고 만다.
        이후 인터뷰 자리에서 우연찮게 다시 만나게 되는데...

        윤서와 수찬, 두 사람의 불편한 만남은 조금씩 서로를 건드린다.
        """,
        "genre": "드라마",
        "actors": ["임선우", "김명찬", "이장유", "박현숙"]
    },
    {
        "synopsis": """
        "선생님, 저랑 사귀실래요?" 적극적인 어른 민주
        "꺼져" 철벽 많은 급식 윤서

        윤서는 학교에서 학생들에게 인기가 매우 많은 선생님이다.
        어느 날, 윤서는 민주로부터 고백을 받게 된다.
        하지만 윤서는 민주를 거절한다.

        윤서와 민주, 두 사람의 아슬아슬한 만남은 계속된다.
        """,
        "genre": "로맨스",
        "actors": ["김민주", "박서준", "이도현"]
    },
    {
       "synopsis": """
        1919년, 3.1 운동 이후 봉오동 전투에서 승리한 독립군의 이야기를 그린 영화.
        """,
        "genre": "액션",
        "actors": ["유해진", "류준열", "조우진"]
    }
]

# 가중치
genre_weight = 3
actor_weight = 3

# 각 영화별 전처리된 토큰 리스트 생성 (2차원 배열)
documents_tokens = [
    preprocess_synopsis(movie["synopsis"], movie["genre"], movie["actors"], genre_weight, actor_weight)
    for movie in movies
]

# Word2Vec 하이퍼파라미터
vector_size = 100  # 임베딩 벡터 차원
window = 5  # 윈도우 크기 (주변 단어 범위)
min_counts = [1, 2, 3]  # 최소 단어 빈도 (변경하면서 실험)
sg_values = [0, 1]  # 0: CBOW, 1: Skip-gram

def calculate_movie_vector(model, tokens):
    """
    영화별 토큰들의 평균 벡터를 계산하는 함수.
    
    Args:
      model: Word2Vec 모델
      tokens: 영화별 토큰 리스트
      
    Returns:
      영화 벡터 (NumPy 배열)
    """
    
    word_vectors = []
    for token in tokens:
      if token in model.wv:
        word_vectors.append(model.wv[token])
        
    if len(word_vectors) > 0:
      return np.mean(word_vectors, axis=0) # 모든 단어 벡터의 평균
    else:
      return np.zeros(model.vector_size) # 빈 벡터 반환 (해당 영화에 모델 vocabulary에 있는 단어가 없는 경우)
    

# 여러 min_count와 sg 값에 대해 실험
for min_count in min_counts:
    for sg in sg_values:
        print(f"\n--- min_count: {min_count}, sg: {sg} ({'CBOW' if sg == 0 else 'Skip-gram'}) ---")

        # Word2Vec 모델 학습
        model = Word2Vec(
            sentences=documents_tokens,
            vector_size=vector_size,
            window=window,
            min_count=min_count,
            sg=sg,  # 0: CBOW, 1: Skip-gram
            workers=4,  # 병렬 처리 스레드 수
        )

        # 영화별 토큰 평균 벡터 계산
        movie_vectors = [calculate_movie_vector(model, tokens) for tokens in documents_tokens]

        # 영화 간 코사인 유사도 계산
        similarity_matrix = cosine_similarity(movie_vectors)
        print("코사인 유사도 행렬:\n", similarity_matrix)

        # 0번 영화와 다른 영화 간의 유사도 확인
        print(f"0번 영화와 다른 영화 간 유사도: {similarity_matrix[0][1:]}")
```

**설명:**

1.  **`Word2Vec` 모델 학습:**
    *   `sentences`: 토큰화된 문서 리스트(2차원 배열)를 입력으로 받습니다.
    *   `vector_size`: 임베딩 벡터의 차원을 지정합니다.
    *   `window`: 윈도우 크기(주변 단어 고려 범위)를 지정합니다.
    *   `min_count`: 지정된 빈도보다 적게 나타나는 단어는 무시합니다.
    *   `sg`: 0이면 CBOW, 1이면 Skip-gram 모델을 사용합니다.
    *   `workers`: 학습에 사용할 스레드 수를 지정합니다.

2.  **`calculate_movie_vector` 함수 :**
    *   `word2vec.wv[token]`: Word2Vec 모델의 `wv` 속성을 사용하여 각 토큰의 임베딩 벡터를 가져옵니다.
    *    `word_vectors` 리스트에 각 단어 벡터를 추가한다.
    *   `np.mean(word_vectors, axis=0)`: 영화에 속한 모든 단어 벡터의 평균을 계산하여 영화 벡터를 생성합니다.  `axis=0`은 열(column) 방향으로 평균을 계산하라는 의미입니다.
    *   `model.wv`에 없는 단어는 무시합니다.
    *   해당 영화의 어떤 단어도 `model.wv`에 없다면 0으로 채워진 벡터를 반환합니다.

3.  **영화별 토큰 평균 벡터 계산:**
    *   각 영화의 토큰 리스트에 대해 `calculate_movie_vector`함수를 호출하여 영화 벡터를 계산합니다.

4.  **코사인 유사도 계산:**
    *   `cosine_similarity` 함수를 사용하여 영화 벡터 간의 코사인 유사도 행렬을 계산합니다.

5.  **`min_count`와 `sg` 변경:**
    *   `min_count`와 `sg` 값을 변경하면서 Word2Vec 모델을 학습시키고, 각 경우에 대한 코사인 유사도 행렬을 출력합니다.

**결과 해석:**

*   `min_count`를 늘리면, 더 자주 등장하는 단어들만 고려하게 되므로, 덜 중요한 단어의 영향을 줄일 수 있습니다. 하지만 너무 높게 설정하면 중요한 단어도 제외될 수 있습니다.
*   `sg` 값을 변경하면 CBOW와 Skip-gram 모델 간의 성능 차이를 비교할 수 있습니다.
*   코사인 유사도 행렬을 통해 영화 간의 유사도를 확인할 수 있습니다. 1에 가까울수록 유사하고, 0에 가까울수록 관련이 없습니다.

**주의:**

*   Word2Vec은 단어의 의미를 학습하지만, 문맥(context)을 완벽하게 이해하지는 못합니다.
*   작은 데이터셋에서는 Word2Vec의 성능이 제한적일 수 있습니다. 더 큰 코퍼스(corpus)로 사전 학습된(pre-trained) Word2Vec 모델을 사용하는 것이 좋습니다. (예: Google News Vectors)
